In [1]:
import hail as hl

In [2]:
rg37 = hl.get_reference('GRCh37')
rg38 = hl.get_reference('GRCh38')

Initializing Spark and Hail with default parameters...
Running on Apache Spark version 2.4.3
SparkUI available at http://nb1-m.c.genotype-qc-neale-lab.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.19-c6ec8b76eb26
LOGGING: writing to /home/hail/hail-20190823-1140-0.2.19-c6ec8b76eb26.log


In [3]:
rg38.add_liftover('gs://hail-common/references/grch38_to_grch37.over.chain.gz', rg37) 

## Perform GRCh38 -> GRCh37 liftover for SPARK

In [27]:
wd = 'gs://qc-nbaya/spark/array_May2019/'
bfile = wd+'SPARK.27K.genotype.20190501'

In [6]:
mt = hl.methods.import_plink(bed=bfile+'.bed',
                            bim=bfile+'.bim',
                            fam=bfile+'.fam',
                            reference_genome=rg38)

2019-08-01 20:48:14 Hail: INFO: Found 27099 samples in fam file.
2019-08-01 20:48:14 Hail: INFO: Found 632015 variants in bim file.


In [5]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'fam_id': str
    'pat_id': str
    'mat_id': str
    'is_female': bool
    'is_case': bool
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'cm_position': float64
----------------------------------------
Entry fields:
    'GT': call
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [17]:
mt = mt.annotate_rows(locus_rg37 = hl.liftover(x=mt.locus,
                                               dest_reference_genome='GRCh37',
                                              include_strand=True))

In [18]:
mt.aggregate_rows(hl.agg.stats(mt.locus_rg37.is_negative_strand))

2019-08-01 20:59:53 Hail: INFO: Ordering unsorted dataset with network shuffle


Struct(mean=8.385916723100193e-05, stdev=0.009157081132712172, min=0.0, max=1.0, n=632012, sum=52.999999999999986)

In [22]:
mt1 = mt.filter_rows(hl.is_defined(mt.locus_rg37))
print(mt1.count_rows())

2019-08-01 21:02:54 Hail: INFO: Ordering unsorted dataset with network shuffle


632012


In [23]:
mt2 = mt1.filter_rows(~mt1.locus_rg37.is_negative_strand)
print(mt2.count_rows())

2019-08-01 21:03:12 Hail: INFO: Ordering unsorted dataset with network shuffle


631959


In [38]:
mt2 = mt2.annotate_rows(new_locus = mt2.locus_rg37.result)
mt3 = mt2.key_rows_by(*[mt2.new_locus,mt2.alleles])

In [39]:
mt4 = mt3.drop('locus')
mt4 = mt4.rename({'new_locus':'locus'})

In [43]:
mt4.count_rows()

2019-08-01 21:50:15 Hail: INFO: Ordering unsorted dataset with network shuffle


631959

In [40]:
mt4.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'fam_id': str
    'pat_id': str
    'mat_id': str
    'is_female': bool
    'is_case': bool
----------------------------------------
Row fields:
    'alleles': array<str>
    'rsid': str
    'cm_position': float64
    'locus_rg37': struct {
        result: locus<GRCh37>, 
        is_negative_strand: bool
    }
    'locus': locus<GRCh37>
----------------------------------------
Entry fields:
    'GT': call
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [41]:
hl.export_plink(dataset=mt4, 
                output=wd+'SPARK.27K.genotype.20190501.liftover',
                call=mt4.GT,
                fam_id=mt4.fam_id,
                ind_id=mt4.s,
                pat_id=mt4.pat_id,
                mat_id=mt4.mat_id,
                is_female=mt4.is_female,
                pheno=mt4.is_case,
                varid=mt4.rsid,
                cm_position = mt4.cm_position)

2019-08-01 21:20:50 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-01 21:20:59 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-01 21:21:06 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-01 21:21:13 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-01 21:23:47 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-01 21:29:45 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-01 21:41:36 Hail: INFO: merging 33 files totalling 4.0G...
2019-08-01 21:42:51 Hail: INFO: while writing:
    gs://qc-nbaya/spark/array_May2019/SPARK.27K.genotype.20190501.liftover.bed
  merge time: 1m14.9s
2019-08-01 21:42:51 Hail: INFO: merging 32 files totalling 19.6M...
2019-08-01 21:42:52 Hail: INFO: while writing:
    gs://qc-nbaya/spark/array_May2019/SPARK.27K.genotype.20190501.liftover.bim
  merge time: 1.320s
2019-08-01 21:42:54 Hail: INFO: merging 4 files totalling 915.5K...
2019-08-01 21:42:55 Hail: IN

In [42]:
mt4.rsid.show()

2019-08-01 21:43:10 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-01 21:43:24 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-01 21:43:38 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-01 21:43:47 Hail: INFO: Ordering unsorted dataset with network shuffle


locus,alleles,rsid
locus<GRCh37>,array<str>,str
1:565508,"[""A"",""G""]","""GSA-rs9283150"""
1:726912,"[""A"",""G""]","""GSA-1:726912"""
1:727841,"[""G"",""A""]","""GSA-rs116587930"""
1:752721,"[""G"",""A""]","""rs3131972"""
1:756268,"[""A"",""G""]","""rs12567639"""
1:759036,"[""G"",""A""]","""GSA-rs114525117"""
1:794332,"[""G"",""A""]","""rs12127425"""
1:801536,"[""T"",""G""]","""GSA-rs79373928"""
1:815421,"[""C"",""T""]","""GSA-rs72888853"""


## Perform GRCh38 –> GRCh37 liftover for HGDP
HGDP is already filtered to SPARK rsids

In [5]:
wd = 'gs://qc-nbaya/spark/array_May2019/'
bfile = wd+'hgdp_wgs.spark.allchr'

In [6]:
mt = hl.methods.import_plink(bed=bfile+'.bed',
                            bim=bfile+'.bim',
                            fam=bfile+'.fam',
                            reference_genome=rg38)

2019-08-23 11:52:08 Hail: INFO: Found 929 samples in fam file.
2019-08-23 11:52:08 Hail: INFO: Found 571831 variants in bim file.


In [7]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'fam_id': str
    'pat_id': str
    'mat_id': str
    'is_female': bool
    'is_case': bool
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'cm_position': float64
----------------------------------------
Entry fields:
    'GT': call
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [8]:
mt = mt.annotate_rows(locus_rg37 = hl.liftover(x=mt.locus,
                                               dest_reference_genome='GRCh37',
                                               include_strand=True))

In [10]:
mt.aggregate_rows(hl.agg.stats(mt.locus_rg37.is_negative_strand))

2019-08-23 11:53:58 Hail: INFO: Coerced sorted dataset
2019-08-23 11:53:58 Hail: INFO: Coerced sorted dataset


Struct(mean=1.748771488029659e-06, stdev=0.0013224100838347922, min=0.0, max=1.0, n=571830, sum=1.0)

In [11]:
mt1 = mt.filter_rows(hl.is_defined(mt.locus_rg37))
print(mt1.count_rows())

2019-08-23 11:54:14 Hail: INFO: Coerced sorted dataset


571830


In [12]:
mt2 = mt1.filter_rows(~mt1.locus_rg37.is_negative_strand)
print(mt2.count_rows())

2019-08-23 11:54:22 Hail: INFO: Coerced sorted dataset


571829


In [13]:
mt2 = mt2.annotate_rows(new_locus = mt2.locus_rg37.result)
mt3 = mt2.key_rows_by(*[mt2.new_locus,mt2.alleles])

In [14]:
mt4 = mt3.drop('locus')
mt4 = mt4.rename({'new_locus':'locus'})

In [17]:
hl.export_plink(dataset=mt4, 
                output=bfile+'.GRCh37',
                call=mt4.GT,
                fam_id=mt4.fam_id,
                ind_id=mt4.s,
                pat_id=mt4.pat_id,
                mat_id=mt4.mat_id,
                is_female=mt4.is_female,
                pheno=mt4.is_case,
                varid=mt4.rsid,
                cm_position = mt4.cm_position)

2019-08-23 11:58:52 Hail: INFO: Coerced sorted dataset
2019-08-23 11:58:56 Hail: INFO: Coerced sorted dataset
2019-08-23 11:59:01 Hail: INFO: Coerced sorted dataset
2019-08-23 11:59:06 Hail: INFO: Coerced sorted dataset
2019-08-23 11:59:27 Hail: INFO: Coerced sorted dataset
2019-08-23 11:59:49 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-23 12:01:40 Hail: INFO: merging 3 files totalling 127.1M...
2019-08-23 12:01:52 Hail: INFO: while writing:
    gs://qc-nbaya/spark/array_May2019/gs://qc-nbaya/spark/array_May2019/hgdp_wgs.spark.allchr.GRCh37.bed
  merge time: 12.712s
2019-08-23 12:01:59 Hail: INFO: merging 2 files totalling 16.6M...
2019-08-23 12:02:03 Hail: INFO: while writing:
    gs://qc-nbaya/spark/array_May2019/gs://qc-nbaya/spark/array_May2019/hgdp_wgs.spark.allchr.GRCh37.bim
  merge time: 3.624s
2019-08-23 12:02:04 Hail: INFO: merging 4 files totalling 21.8K...
2019-08-23 12:02:05 Hail: INFO: while writing:
    gs://qc-nbaya/spark/array_May2019/gs://qc-nbay